### Type 1: Non-personalized recommendations
Based on review score and count

### Type 2: Collaborative Filtering
- User-based, which measures the similarity between target users and other users.
- Item-based, which measures the similarity between the items that target users rate or interact with and other items.

In [6]:
import numpy as np; 
import pandas as pd; 
from pandas import Series, DataFrame

In [7]:
rating = pd.read_csv('data/movie_rating.csv')
rating.head()

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0


In [9]:
rp = rating.pivot_table(index=['title'], columns=['critic'], values='rating')
rp

critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
title,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


In [10]:
rating_toby = rp['Toby']
sim_toby = rp.corrwith(rating_toby)
sim_toby

critic
Claudia Puig     0.893405
Gene Seymour     0.381246
Jack Matthews    0.662849
Lisa Rose        0.991241
Mick LaSalle     0.924473
Toby             1.000000
dtype: float64

In [11]:
rating_c = rating.loc[rating_toby[rating.title].isnull().values & (rating.critic != 'Toby')]
rating_c

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
4,Jack Matthews,The Night Listener,3.0
5,Mick LaSalle,Lady in the Water,3.0
7,Mick LaSalle,Just My Luck,2.0
10,Mick LaSalle,The Night Listener,3.0
12,Claudia Puig,Just My Luck,3.0
15,Claudia Puig,The Night Listener,4.5
16,Lisa Rose,Lady in the Water,2.5
18,Lisa Rose,Just My Luck,3.0
20,Lisa Rose,The Night Listener,3.0


In [12]:
rating_c_similarity = rating_c['critic'].map(sim_toby)
rating_c_similarity

0     0.662849
4     0.662849
5     0.924473
7     0.924473
10    0.924473
12    0.893405
15    0.893405
16    0.991241
18    0.991241
20    0.991241
25    0.381246
27    0.381246
30    0.381246
Name: critic, dtype: float64

In [16]:
rating_c = rating_c.assign(similarity=rating_c_similarity,
                           sim_rating=rating_c_similarity * rating_c.rating)
rating_c.sort_values('title')

,critic,title,rating,similarity,sim_rating
7,Mick LaSalle,Just My Luck,2.0,0.924473,1.848947
12,Claudia Puig,Just My Luck,3.0,0.893405,2.680215
18,Lisa Rose,Just My Luck,3.0,0.991241,2.973722
27,Gene Seymour,Just My Luck,1.5,0.381246,0.571870
0,Jack Matthews,Lady in the Water,3.0,0.662849,1.988547
5,Mick LaSalle,Lady in the Water,3.0,0.924473,2.773420
16,Lisa Rose,Lady in the Water,2.5,0.991241,2.478102
25,Gene Seymour,Lady in the Water,3.0,0.381246,1.143739
4,Jack Matthews,The Night Listener,3.0,0.662849,1.988547
10,Mick LaSalle,The Night Listener,3.0,0.924473,2.773420


In [14]:
recommendations = rating_c.groupby('title').apply(lambda s: s.sim_rating.sum() / s.similarity.sum())
recommendations.sort_index(ascending=False)

title
The Night Listener    3.347790
Lady in the Water     2.832550
Just My Luck          2.530981
dtype: float64